# RNN_순환신경망_텍스트분석
* 언어, 시계열 데이터 분석에 주로 사용
* 길이가 길어지면 기울기 소실문제 발생
* LSTM, GRU 등으로 문제를 보완

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding
from tensorflow.keras.utils import to_categorical 

import pandas as pd
import numpy as np

## 텐서플로에서 텍스트 전처리하기 
* 토큰화 : 문장을 단어 혹은 형태소로 쪼개는 것
* 원핫인코딩 : 문자를 벡터화
* 임베딩(embedding) : 벡터화 (원핫인코딩이랑은 조금 다름) --> 원핫인코딩을 더 축소 

- 원핫인코딩: 범주 수만큼 차원이 늘어나며 각 벡터는 이진 값으로 표현됨.
- 임베딩: 저차원의 실수 벡터로 표현하며, 학습 과정을 통해 벡터 간의 유사성을 학습함.

<br><br>

1. 원핫인코딩 (One-Hot Encoding)
- 개념: 범주형 데이터를 이진 벡터로 변환하는 방법입니다.
- 방법: 각 범주를 0과 1로만 이루어진 벡터로 변환합니다. 예를 들어, 세 가지 범주가 있다면 각 범주는 [1, 0, 0], [0, 1, 0], [0, 0, 1]로 표현됩니다.

- 특징:
    - 벡터의 길이는 범주의 수와 동일합니다.
    - 벡터 사이의 유사성을 표현하지 않기 때문에 범주 간의 관계나 순서 정보를 반영하지 못합니다.
    - 차원의 저주(Dimension Curse) 문제가 발생할 수 있습니다. 즉, 범주가 많아질수록 벡터의 길이가 매우 길어집니다.
- 장점: 간단하고 빠르게 구현 가능.
- 단점: 차원이 매우 커질 수 있고, 메모리 비효율적.

<br>
2. 임베딩 (Embedding)  <br>
- 개념: 범주형 데이터를 고차원의 실수 벡터로 매핑하는 방법으로, 주로 딥러닝에서 사용됩니다.
- 방법: 각 범주를 고정된 크기의 실수 벡터로 표현하며, 이 벡터들은 학습을 통해 최적화됩니다. 벡터 간의 거리는 범주 간의 유사성을 나타낼 수 있습니다.

- 특징:
    - 저차원 벡터로 범주형 데이터를 표현할 수 있어 메모리와 계산 효율성이 높습니다.
    - 벡터 간의 관계가 학습되므로 범주 간의 유사성이나 의미적인 관계를 학습할 수 있습니다.
    - 주로 딥러닝 모델의 초기 레이어에서 사용되며, 텍스트 데이터를 처리할 때 많이 사용됩니다.
- 장점: 범주 간의 관계를 반영할 수 있고, 원핫인코딩보다 효율적입니다.
- 단점: 학습이 필요하며, 구현이 상대적으로 복잡합니다.

In [4]:
text = "해보지 않으면 해낼 수 없다"

In [5]:
result = text_to_word_sequence(text)
result

['해보지', '않으면', '해낼', '수', '없다']

### 단어 빈도수 세기 

In [15]:
docs = ['먼저 텍스트의 각 단어를 나누어 토큰화합니다', 
        '텍스트의 단어로 토큰화해야 딥러닝에서 인식됩니다', 
        '토큰화한 결과는 딥러닝에서 사용할 수 있습니다',
        '텍스트 전처리에는 벡터화 원핫인코딩 패딩으로 길이 맞추기 등이 필요합니다',
        '딥러닝 쉽지 않네요']

In [16]:
docs[2]

'토큰화한 결과는 딥러닝에서 사용할 수 있습니다'

In [17]:
token = Tokenizer()
token.fit_on_texts(docs)
print("단어 카운트:\n ", token.word_counts)
print()
print("문장 카운트:\n ", token.document_count)
print()
print("각 단어가 몇 개의 문장에 포함되어 있는지 계산:\n ", token.word_docs)
print()
print("각 단어에 매겨진 인덱스 값:\n ", token.word_index)

단어 카운트:
  OrderedDict([('먼저', 1), ('텍스트의', 2), ('각', 1), ('단어를', 1), ('나누어', 1), ('토큰화합니다', 1), ('단어로', 1), ('토큰화해야', 1), ('딥러닝에서', 2), ('인식됩니다', 1), ('토큰화한', 1), ('결과는', 1), ('사용할', 1), ('수', 1), ('있습니다', 1), ('텍스트', 1), ('전처리에는', 1), ('벡터화', 1), ('원핫인코딩', 1), ('패딩으로', 1), ('길이', 1), ('맞추기', 1), ('등이', 1), ('필요합니다', 1), ('딥러닝', 1), ('쉽지', 1), ('않네요', 1)])

문장 카운트:
  5

각 단어가 몇 개의 문장에 포함되어 있는지 계산:
  defaultdict(<class 'int'>, {'나누어': 1, '토큰화합니다': 1, '먼저': 1, '각': 1, '단어를': 1, '텍스트의': 2, '토큰화해야': 1, '단어로': 1, '인식됩니다': 1, '딥러닝에서': 2, '사용할': 1, '결과는': 1, '토큰화한': 1, '있습니다': 1, '수': 1, '길이': 1, '원핫인코딩': 1, '전처리에는': 1, '필요합니다': 1, '등이': 1, '텍스트': 1, '벡터화': 1, '맞추기': 1, '패딩으로': 1, '쉽지': 1, '않네요': 1, '딥러닝': 1})

각 단어에 매겨진 인덱스 값:
  {'텍스트의': 1, '딥러닝에서': 2, '먼저': 3, '각': 4, '단어를': 5, '나누어': 6, '토큰화합니다': 7, '단어로': 8, '토큰화해야': 9, '인식됩니다': 10, '토큰화한': 11, '결과는': 12, '사용할': 13, '수': 14, '있습니다': 15, '텍스트': 16, '전처리에는': 17, '벡터화': 18, '원핫인코딩': 19, '패딩으로': 20, '길이': 21, '맞추기': 22, '등이': 23, '필요합니다': 24,

ex) " 텍스트의 단어로 토큰화해야 딥러닝에서 인식됩니다 "


텍스트의 / 단어로 / 토큰화해야 / 딥러닝에서 / 인식됩니다
#### 1 8 9 2 10

- 1번에 1
- 2번에 1
- 8번에 1
- 9번에 1
- 10번에 1


110000011100000

#### 1번부터 15번까지 벡터화했을 때

In [19]:
x = token.texts_to_sequences(docs)
print(x)

# 문장의 벡터화 

[[3, 1, 4, 5, 6, 7], [1, 8, 9, 2, 10], [11, 12, 2, 13, 14, 15], [16, 17, 18, 19, 20, 21, 22, 23, 24], [25, 26, 27]]


In [24]:
# 문장의 길이를 맞추기 위한 패딩
padded_x = pad_sequences(x, 6)
padded_x 


# truncating: String, "pre" or "post" (optional, defaults to `"pre"`)
# remove values from sequences larger than `maxlen`, either at the beginning or at the end of the sequences.
# 네번째 문장의 인덱스가 짤린 이유 : 기본 디폴트 값이 pre로 되어있기 때문이다. 

array([[ 3,  1,  4,  5,  6,  7],
       [ 0,  1,  8,  9,  2, 10],
       [11, 12,  2, 13, 14, 15],
       [19, 20, 21, 22, 23, 24],
       [ 0,  0,  0, 25, 26, 27]], dtype=int32)

In [26]:
lenofsent = []

for i in x:
    #print(len(i))
    lenofsent.append(len(i))
max(lenofsent)

9

In [27]:
# 문장의 길이를 맞추기 위한 패딩
# 가장 긴 문장 길이 + 1 의 길이로 패딩
# 문장 시작에는 0이 있어야 함

padded_x = pad_sequences(x, max([len(i) for i in x]) + 1)
padded_x 


array([[ 0,  0,  0,  0,  3,  1,  4,  5,  6,  7],
       [ 0,  0,  0,  0,  0,  1,  8,  9,  2, 10],
       [ 0,  0,  0,  0, 11, 12,  2, 13, 14, 15],
       [ 0, 16, 17, 18, 19, 20, 21, 22, 23, 24],
       [ 0,  0,  0,  0,  0,  0,  0, 25, 26, 27]], dtype=int32)

## 텍스트를 읽고 긍정 / 부정 예측하기 

In [28]:
docs2 = ["너무 재밌네요",
         "최고예요", 
         "참 신기한 딥러닝이네요", 
         "인공지능 칭찬합니다",
         "더 자세히 배우고 싶어요",
         "변화가 너무 빨라요",
         "GPT성능이 생각보다 별로네요", 
         "Gemini보다는 낫죠",
         "나는 차라리 라마를 쓴다",
         "유료 결재 싫어요"]

In [29]:
classes = np.array([1,1,1,1,1,0,0,1,0,0])

In [30]:
token = Tokenizer()
token.fit_on_texts(docs2)
print(token.word_index)

# 인덱스 출력

{'너무': 1, '재밌네요': 2, '최고예요': 3, '참': 4, '신기한': 5, '딥러닝이네요': 6, '인공지능': 7, '칭찬합니다': 8, '더': 9, '자세히': 10, '배우고': 11, '싶어요': 12, '변화가': 13, '빨라요': 14, 'gpt성능이': 15, '생각보다': 16, '별로네요': 17, 'gemini보다는': 18, '낫죠': 19, '나는': 20, '차라리': 21, '라마를': 22, '쓴다': 23, '유료': 24, '결재': 25, '싫어요': 26}


In [32]:
x = token.texts_to_sequences(docs2)
print("토큰화 결과:\n", x)

토큰화 결과:
 [[1, 2], [3], [4, 5, 6], [7, 8], [9, 10, 11, 12], [13, 1, 14], [15, 16, 17], [18, 19], [20, 21, 22, 23], [24, 25, 26]]


In [33]:
# 문장의 길이를 맞추기 위한 패딩
# 가장 긴 문장 길이 + 1 의 길이로 패딩
# 문장 시작에는 0이 있어야 함

padding_x = pad_sequences(x, max([len(i) for i in x]))   # 4개가 제일 길다 
padding_x 


array([[ 0,  0,  1,  2],
       [ 0,  0,  0,  3],
       [ 0,  4,  5,  6],
       [ 0,  0,  7,  8],
       [ 9, 10, 11, 12],
       [ 0, 13,  1, 14],
       [ 0, 15, 16, 17],
       [ 0,  0, 18, 19],
       [20, 21, 22, 23],
       [ 0, 24, 25, 26]], dtype=int32)

## 임베딩

In [34]:
# 임베딩 
word_size = len(token.word_index) + 1  
word_size

27

In [35]:
model = Sequential()
model.add(Embedding(word_size, 8, input_length=4))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.summary()

2024-09-13 15:03:33.458158: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-13 15:03:33.459958: I tensorflow/c/logging.cc:34] DirectML: creating device on adapter 0 (AMD Radeon(TM) Graphics)
2024-09-13 15:03:33.851638: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-09-13 15:03:33.851689: W tensorflow/core/common_runtime/pluggable_device/pluggable_device_bfc_allocator.cc:28] Overriding allow_growth setting because force_memory_growth was requested by the device.
2024-09-13 15:03:33.851715: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_f

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 8)              216       
                                                                 
 flatten (Flatten)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 249
Trainable params: 249
Non-trainable params: 0
_________________________________________________________________


In [36]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])
model.fit(padding_x, classes, epochs=20)
print(model.evaluate(padding_x, classes)[1])

Epoch 1/20


2024-09-13 15:07:20.581349: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-09-13 15:07:20.656875: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-09-13 15:07:20.656944: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 14845 MB memory) -> physical PluggableDevice (device: 0, name: DML, pci bus id: <undefined>)
2024-09-13 15:07:20.661307: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-09-13 15:07:20.661372: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_f

1/1 [==============================] - 1s 571ms/step - loss: 0.6936 - accuracy: 0.5000
Epoch 2/20
1/1 [==============================] - 0s 13ms/step - loss: 0.6908 - accuracy: 0.5000
Epoch 3/20
1/1 [==============================] - 0s 15ms/step - loss: 0.6882 - accuracy: 0.6000
Epoch 4/20
1/1 [==============================] - 0s 15ms/step - loss: 0.6855 - accuracy: 0.6000
Epoch 5/20
1/1 [==============================] - 0s 15ms/step - loss: 0.6828 - accuracy: 0.7000
Epoch 6/20
1/1 [==============================] - 0s 12ms/step - loss: 0.6801 - accuracy: 0.7000
Epoch 7/20
1/1 [==============================] - 0s 15ms/step - loss: 0.6774 - accuracy: 0.7000
Epoch 8/20
1/1 [==============================] - 0s 15ms/step - loss: 0.6747 - accuracy: 0.8000
Epoch 9/20
1/1 [==============================] - 0s 15ms/step - loss: 0.6721 - accuracy: 0.8000
Epoch 10/20
1/1 [==============================] - 0s 15ms/step - loss: 0.6694 - accuracy: 0.9000
Epoch 11/20
1/1 [======================

2024-09-13 15:07:21.349130: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-09-13 15:07:21.375190: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-09-13 15:07:21.375249: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 14845 MB memory) -> physical PluggableDevice (device: 0, name: DML, pci bus id: <undefined>)
